In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import namedtuple

In [2]:
Planning = pd.read_csv('Planning.csv', sep=';',index_col='Time')
cols = Planning.columns
for col in cols:
    Planning[col] = Planning[col].astype(float)
display(Planning)

,p1,p2,p3
Time,,,
t1,28.732,28.733,28.734
t2,34.229,34.230,34.231
t3,31.676,0.000,31.678
t4,32.022,32.023,32.024
t5,0.000,0.000,0.000
t6,0.000,0.000,0.000
t7,0.000,0.000,0.000
t8,0.000,0.000,20.858
t9,0.000,0.000,0.000


In [41]:
noueds = ['s','j1','j2','r1','r2','r3','r4']
lines = [
    ('s','j1'),
    ('j1','r1'),
    ('j1','r4'),
    ('j1','j2'),
    ('j2','r2'),
    ('j2','r3')
]

r = ['r1','r2','r3','r4']
t = ['t1','t2','t3','t4','t5','t6','t7','t8','t9']

height = {
    's': 0,
    'j1': 30,
    'j2': 30,
    'r1': 50,
    'r2': 50,
    'r3': 45,
    'r4': 45,
}

surface = {
    'r1': 80,
    'r2': 80,
    'r3': 80,
    'r4': 80,
}

vmin = {
    'r1': 100,
    'r2': 100,
    'r3': 100,
    'r4': 100,    
}

vmax = {
    'r1': 300,
    'r2': 300,
    'r3': 300,
    'r4': 300,    
}

tariffnight = 0.02916
tariffday =0.04609


demand = pd.DataFrame(
    np.array([[9.83, 5.0,3.67, 6.5,5.67,7.5,3.0,3.0,2.0],
    [44.83,18.0,0.0, 0.0, 0.0, 0.0,0.0,45.0,51.67],
    [14.0,13.33,25.5,11.0,10.0,10.0,11.0,10.33,30.17],
    [1,1,8.5,9.5,4,2.33,0,1,0.83]]),
    index = r,
    columns = t)

Coeff_Perts = pd.DataFrame(
    np.array([[0.00005425,0.00038190],
          [0.00027996,0.00149576],
           [0.00089535,0.00340436],
           [0.00044768,0.00170218],
           [0.00223839,0.00851091],
          [0.00134303,0.00510655]]),
    index = lines,
    columns = [2,1]).T;

Coeff_Pompes = {
    0:63.0796 ,
    2:-0.0064085
}

Coeff_Puissance = {
    0:3.81101 ,
    1: 0.09627
}

In [4]:
Debit_pipes = pd.DataFrame(index = lines, columns = t).T
Pression_noeuds = pd.DataFrame(index = noueds, columns = t).T
Niveau_reservoirs = pd.DataFrame(index = r, columns = t).T

In [22]:
Planning

,p1,p2,p3
Time,,,
t1,28.732,28.733,28.734
t2,34.229,34.230,34.231
t3,31.676,0.000,31.678
t4,32.022,32.023,32.024
t5,0.000,0.000,0.000
t6,0.000,0.000,0.000
t7,0.000,0.000,0.000
t8,0.000,0.000,20.858
t9,0.000,0.000,0.000


In [12]:
Planning.values.sum(axis = 1)

array([ 86.199, 102.69 ,  63.354,  96.069,   0.   ,   0.   ,   0.   ,
        20.858,   0.   ])

In [38]:
Debit_pipes[('s','j1')] = Planning.values.sum(axis = 1)
Pression_noeuds['s'] = Coeff_Pompes[2]*Planning['p1']**2 + Coeff_Pompes[0]
Pression_noeuds['j1'] = height['s']-height['j1']+Pression_noeuds['s'] - Pertes(Debit_pipes,('s','j1'))

In [68]:
line = ('j1','r1')
time = 't1'
a = Coeff_Perts[line][2]
b = Coeff_Perts[line][1] + 1/surface['r1']
c = -demand[time]['r1']+vmin['r1']/surface['r1']-Pression_noeuds['j1'][time]+height['r1']-height['j1']
Debit_pipes[('j1','r1')][time] = (-b+np.sqrt(b**2-4*a*c))/2*a

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0.13430751555437412

In [36]:
def Pertes(Q, line):
    return Coeff_Perts[line][1]*Q[line]+ Coeff_Perts[line][2]*Q**2

In [ ]:
for line in lines:
    IN, OUT = line
    Pression_noeuds(OUT) = height(IN)+Pression_noeuds(IN)-height(OUT) - Pertes(Debit_pipes[line])

In [14]:
Debit_pipes

,"(s, j1)","(j1, r1)","(j1, r4)","(j1, j2)","(j2, r2)","(j2, r3)"
t1,86.199,NaN,NaN,NaN,NaN,NaN
t2,102.690,NaN,NaN,NaN,NaN,NaN
t3,63.354,NaN,NaN,NaN,NaN,NaN
t4,96.069,NaN,NaN,NaN,NaN,NaN
t5,0.000,NaN,NaN,NaN,NaN,NaN
t6,0.000,NaN,NaN,NaN,NaN,NaN
t7,0.000,NaN,NaN,NaN,NaN,NaN
t8,20.858,NaN,NaN,NaN,NaN,NaN
t9,0.000,NaN,NaN,NaN,NaN,NaN
